# 自定义层

深度学习成功的一个因素
是提供了各种各样的层
这些层可以以创造性的方式组合
设计出适合各种任务的架构。
例如，研究人员发明了专门处理图像、文本、
循环遍历序列数据
以及
执行动态规划的层。
迟早你会需要
一个在深度学习框架中尚不存在的层。
在这种情况下，你必须构建一个自定义层。
在本节中，我们将向你展示如何操作。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

## (**没有参数的层**)

首先，我们构建一个自定义层，
这个层本身没有任何参数。
如果你还记得我们在:numref:`sec_model_construction`中
对模块的介绍，这应该看起来很熟悉。
下面的`CenteredLayer`类只是
从其输入中减去均值。
要构建它，我们只需要继承
基础层类并实现前向传播函数。

In [2]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

让我们通过向其中输入一些数据来验证我们的层是否按预期工作。

In [3]:
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

我们现在可以[**将我们的层作为组件
用于构建更复杂的模型。**]

In [4]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

作为一种额外的检查，我们可以发送随机数据
通过网络并检查其平均值是否确实为0。
因为我们处理的是浮点数，
我们可能仍会看到一个非常小的非零数字
这是由于量化造成的。

In [5]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-6.5193e-09, grad_fn=<MeanBackward0>)

## [**带有参数的层**]

现在我们已经知道如何定义简单的层，
接下来让我们学习如何定义可以通过训练调整参数的层。
我们可以使用内置函数来创建参数，这些函数
提供了一些基本的管理功能。
特别是，它们控制访问、初始化、
共享、保存和加载模型参数。
这样，除了其他好处之外，我们不需要为每个自定义层
编写自定义的序列化例程。

现在让我们实现我们自己的全连接层版本。
回想一下，这一层需要两个参数，
一个表示权重，另一个表示偏置。
在这个实现中，我们默认集成了ReLU激活函数。
该层需要两个输入参数：`in_units` 和 `units`，分别
表示输入和输出的数量。

In [6]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

接下来，我们实例化 `MyLinear` 类并访问其模型参数。

In [7]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.4783,  0.4284, -0.0899],
        [-0.6347,  0.2913, -0.0822],
        [-0.4325, -0.1645, -0.3274],
        [ 1.1898,  0.6482, -1.2384],
        [-0.1479,  0.0264, -0.9597]], requires_grad=True)

我们可以直接使用自定义层进行前向传播计算。

In [8]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.9316, 0.0000],
        [0.1808, 1.4208, 0.0000]])

我们也可以（**使用自定义层构建模型。**）
一旦我们有了它，就可以像使用内置的全连接层一样使用它。

In [9]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[ 0.0000],
        [13.0800]])

## 摘要

我们可以通过基本的层类来设计自定义层。这使我们能够定义与库中任何现有层行为不同的灵活新层。
一旦定义，自定义层可以在任意上下文和架构中被调用。
层可以有局部参数，这些参数可以通过内置函数创建。

## 练习

1. 设计一个层，该层接收输入并计算张量缩减，即返回 $y_k = \sum_{i, j} W_{ijk} x_i x_j$。
1. 设计一个层，该层返回数据的前半部分傅里叶系数。

[讨论](https://discuss.d2l.ai/t/59)